# M115 - Image Analysis and Processing, Assignment 2 (Notebook 1)

---

In this assignment, an intelligent system was developed to detect pneumonia in chest X-ray images, utilising a dataset available at [Kaggle Chest X-Ray Images (Pneumonia) dataset](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia). The task involved the creation of two sorts of algorithms; classical Machine Learning and Deep Learning ones. This notebook covers the Classical ML aspect, and all the preprocessing thereof.


This coursework is submitted as part of the requirements for the Image Analysis and Processing (M115) course during the spring semester of 2023, in the DSIT's Master degree programme at the National and Kapodistrian University of Athens. The author of this project is

- Michael Darmanis (SID: 7115152200004).



Parts of the code presented in the lecture have been used, and when foreign code (or parts of it) is invoked, it is explicitly mentioned.

The notebook was executed in Kaggle, so bear that in mind in case any issues arise while rerunning parts of the code locally (or in Google Colab, for that matter).

Whatever the case may be, it is essential to **run all the necessary libraries (following four code cells) beforehand**, otherwise the instance of matplotlib will not function properly. It is also advised that all cells are run in of appearnce because many exercise are dependent (variable or otherwise) from previously calculated results.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import cv2
from PIL import Image


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


from sklearn.metrics import f1_score, balanced_accuracy_score, precision_score, matthews_corrcoef, recall_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


from yellowbrick.model_selection import learning_curve


from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


from sklearn.utils import shuffle as shf
import pickle
import os
import glob as gb

import cv2
import skimage
from skimage import feature, filters
from tqdm import tqdm

In [ ]:
def show_random_images():
    path_random_normal = random.choice(train_normal)
    path_random_pneumonia = random.choice(train_pneumonia)
    
    fig = plt.figure(figsize=(10, 10))
    
    ax1 = plt.subplot(1, 2, 1)
    ax1.imshow(Image.open(path_random_normal).convert("LA"))
    ax1.set_title("Normal X-ray")
    
    ax2 = plt.subplot(1, 2, 2)
    ax2.imshow(Image.open(path_random_pneumonia).convert("LA"))
    ax2.set_title("Pneumonia X-ray")

    
def print_metrics(y_pred, y_train, yt_pred, y_test):
    print('Train data metrics:')
    print('Balanced accuracy score: ', balanced_accuracy_score(y_train, y_pred))
    print('F1 score: ', f1_score(y_train, y_pred))
    print('Precison: ', precision_score(y_train, y_pred))
    print('Recall: ', recall_score(y_train, y_pred))
    print('MCC',  matthews_corrcoef(y_train, y_pred))
    print()
    print('Test data metrics:')
    print('Balanced accuracy score: ', balanced_accuracy_score(y_test, yt_pred))
    print('F1 score: ', f1_score(y_test, yt_pred))
    print('Precison: ', precision_score(y_test, yt_pred))
    print('Recall: ', recall_score(y_test, yt_pred))
    print('MCC',  matthews_corrcoef(y_test, yt_pred))


#function to plot the confusion matrix for each model
def plot_confusion_matrix(predictions, y_test, title):
    labels = ['Normal', 'Pnuemonia']
    
    cm = confusion_matrix(y_test,predictions)
    cm = pd.DataFrame(cm , index = ['0','1'] , columns = ['0','1'])
    
    plt.figure(figsize = (10, 10))
    plt.title(title)
    sns.heatmap(cm, linecolor = 'black' , linewidth = 1,
                annot = True, fmt='', xticklabels = labels,
                yticklabels = labels)
    plt.show()
    
    
def evaluate_classifiers(X_test, y_test):
    # Create a dictionary to store your classifiers
    classifiers = {'Logistic Regression': log_reg, 'Decision Tree': dtc,
                   'Random Forest': rfc, 'SVM': svm}

    # Initialize an empty dictionary to store the scores for each classifier
    scores = {'Classifier': [], 'F1': [], 'Balanced Accuracy': [], 'Precision': [],
              'MCC': [], 'Recall': []}

    # Iterate over the classifiers
    for clf_name, clf in classifiers.items():
        y_pred = clf.predict(X_test)

        # Calculate scores
        f1 = f1_score(y_test, y_pred, average='weighted') 
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        mcc = matthews_corrcoef(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average='weighted')

        # Add scores to the scores dictionary
        scores['Classifier'].append(clf_name)
        scores['F1'].append(f1)
        scores['Balanced Accuracy'].append(balanced_accuracy)
        scores['Precision'].append(precision)
        scores['MCC'].append(mcc)
        scores['Recall'].append(recall)

    return pd.DataFrame(scores)

### Data exploration

In [ ]:
train_normal = gb.glob("../input/chest-xray-pneumonia/chest_xray/train/NORMAL/*")
train_pneumonia = gb.glob("../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/*")

test_normal = gb.glob("../input/chest-xray-pneumonia/chest_xray/test/NORMAL/*")
test_pneumonia = gb.glob("../input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/*")

val_normal = gb.glob("../input/chest-xray-pneumonia/chest_xray/val/NORMAL/*")
val_pneumonia = gb.glob("../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/*")

In [ ]:
print('Train normal:', len(train_normal), '   ', 'Train pneumonia:', len(train_pneumonia))
print('Test normal:', len(test_normal), '     ', 'Test pneumonia:', len(test_pneumonia))
print('Validation normal:', len(val_normal), ' ', 'Validation pneumonia:', len(val_pneumonia))

First of all, the validation size is insignificant. It will therefore be concatenated with the training data and, should validation data be required, the training data will be randomly split at a 9-1 analogy.

There also exists an imbalance of classes. To mitigate this issue, the `class_weight` parameter will be employed with the value `balanced`, for all ML models used. The `class_weight` argument modifies the loss function during training by assigning a higher penalty for misclassifying the minority class.

In the face of class imbalance, the implementation of metrics that evaluate performance across all classes, also becomes crucial. Therefore the following metrics will be employed throughout:
- Balanced accuracy
- F1 score
- Precision
- Recall
- Matthews correlation coefficient

Balanced Accuracy, which computes the average recall for each class, provides an effective solution for imbalanced datasets due to its indifference towards the majority class. The F1 score, representing the harmonic mean of precision and recall, is also valuable, especially when the positive class bears more significance and a balance between Precision (representing the accuracy of positive predictions) and Recall (indicating the detection rate of all positive instances) is sought.

Additionally, Precision and Recall are key metrics that permit model fine-tuning in specific directions. Precision, the ratio of true positive predictions to the total predicted positives, becomes vital when the reduction of false positives is the goal. Conversely, Recall, the ratio of true positive predictions to all actual positive instances, is pivotal when the maximization of positive instance detection is the objective. The Matthews Correlation Coefficient (MCC), an all-encompassing measure for binary classifications, takes into account both true and false positives and negatives, thereby offering a balanced metric, particularly useful when the classes are of vastly different sizes.

Within the framework of the currect project, thse metrics will provide an appropriate evaluation of the models' performance; permitting necessary adjustments for managing class imbalance.

Next, random images from both train and test sets are checked.

In [ ]:
show_random_images()

In [ ]:
show_random_images()
del train_normal, test_normal, val_normal
del train_pneumonia, test_pneumonia, val_pneumonia

Upon inspection, the images seems to vary in both size and rotation (slightly); this will be born to mind when preprocessing the data and when fitting the ML models.

### On-the-spot attempt

For a first, on-the-spot, attempt; a basic preprocessing will be perfomed. More particularly, the images will be reduced to 300 by 300 pixels and a normalisation & Principal Component Analysis will be applied.

<div style="text-align:center">
<img src="https://i.imgur.com/gW78vT9.png" alt="workflow" width="500" height="600"/>
</div>

Four classifier models will be trained: Linear Regression, Random Forest, Decision Tree, and a Support Vector Machine. The model yielding the best output in terms of the chosen metrics will then be parameter-tuned using a grid search.

In [ ]:
code = {'NORMAL':0 ,'PNEUMONIA':1}
#function to return the class of the images from its number, so the function would return 'Normal' if given 0, and 'PNEUMONIA' if given 1.
def getcode(n) : 
    for x , y in code.items() : 
        if n == y : 
            return x

In [ ]:
# the directories that contain the train and validation images set
paths = ['../input/chest-xray-pneumonia/chest_xray/train/', 
         '../input/chest-xray-pneumonia/chest_xray/val/']

X_train = []
y_train = []

for trainpath in paths:
    for folder in  os.listdir(trainpath) : 
        files = gb.glob(pathname= str( trainpath + folder + '/*.jpeg'))
        for file in files: 
            image = cv2.imread(file)
            #resize images to 300 x 300 pixels
            image_array = cv2.resize(image , (300, 300))
            X_train.append(list(image_array))
            y_train.append(code[folder])

X_train = np.asarray(X_train)
X_train = X_train.astype(np.float32)
np.save('X_train', X_train)
del X_train

y_train = np.asarray(y_train)
y_train = y_train.astype(np.float32)
np.save('y_train', y_train)
del y_train

In [ ]:
#the directory that contain the test images set
testpath='../input/chest-xray-pneumonia/chest_xray/test/'

X_test = []
y_test = []
for folder in  os.listdir(testpath) : 
    files = gb.glob(pathname= str( testpath + folder + '/*.jpeg'))
    for file in files: 
        image = cv2.imread(file)
        #resize images to 300 x 300 pixels
        image_array = cv2.resize(image , (300, 300))
        X_test.append(list(image_array))
        y_test.append(code[folder])

X_test = np.asarray(X_test)
X_test = X_test.astype(np.float32)
np.save('X_test',X_test)
del X_test

y_test = np.asarray(y_test)
y_test = y_test.astype(np.float32)
np.save('y_test',y_test)
del y_test

In [ ]:
#X_train, X_test contain the images as numpy arrays, while y_train, y_test contain the class of each image 
loaded_X_train = np.load('./X_train.npy')
loaded_X_test = np.load('./X_test.npy')
loaded_y_train = np.load('./y_train.npy')
loaded_y_test = np.load('./y_test.npy')

In [ ]:
#flatten the images into a 2d array, for model training and testing
X_train = loaded_X_train.reshape([-1, np.product((300, 300, 3))])
X_test = loaded_X_test.reshape([-1, np.product((300, 300, 3))])
del loaded_X_train, loaded_X_test

In [ ]:
y_train = loaded_y_train
y_test = loaded_y_test
del loaded_y_train, loaded_y_test

In [ ]:
#shuffle train and test data sets in a consistent way
X_train, y_train = shf(X_train, y_train, random_state=15)
X_test, y_test = shf(X_test, y_test, random_state=15)

In [ ]:
#Scaling
sc = StandardScaler(copy=False)
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
#PCA    
pca = PCA(.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
#printing the variance of each component from PCA
print('Number of components after PCA: ' + str(pca.n_components_))

In [ ]:
#making an instance for each algorithm
log_reg  = LogisticRegression(class_weight='balanced')
dtc  = DecisionTreeClassifier(class_weight='balanced')
rfc = RandomForestClassifier(class_weight='balanced')
svm = SVC(class_weight='balanced')

In [ ]:
#fitting each model using X_train and y_train
log_reg.fit(X_train, y_train)
dtc.fit(X_train, y_train)
rfc.fit(X_train, y_train)
svm.fit(X_train, y_train)

In [ ]:
# Convert the dictionary to a pandas DataFrame
df_scores = evaluate_classifiers(X_test, y_test)
print(df_scores)

In [ ]:
mcc_scorer = make_scorer(matthews_corrcoef)

# Defining parameter range
param_grid_svm = {'C': [0.1, 1, 10],  
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf', 'linear', 'poly']} 

grid_svm = GridSearchCV(svm, param_grid_svm, refit = True, verbose = 3, scoring=mcc_scorer)
  
# fitting the model for grid search on the training data
grid_svm.fit(X_train, y_train)

# Inspect the best parameters found by GridSearchCV
print('Best parameters for SVM:', grid_svm.best_params_)

In [ ]:
# Get the best model
best_svm = grid_svm.best_estimator_

# Calculate and print the metrics
print_metrics(best_svm.predict(X_train), y_train, best_svm.predict(X_test), y_test)

In [ ]:
plot_confusion_matrix(best_svm.predict(X_test), y_test, 'Optimised SVM')

In [ ]:
del X_train, y_train, X_test, y_test, pca, sc

# Feature Extraction

As an alternative, and more nuanced approach, traditional image processing methods, with the aim of extracting features for classification tasks, will be applied. This is accomplished by using a subset of available image processing techniques. Here's a brief overview of the steps involved and their importance:

1. **Equalisation**: Image equalisation enhances the contrast of the lungs and accentuates the presence of opacity. This will most likely improve contrast which facilitates subsequent feature extraction.

2. **Image Sharpening**: High pass filtering is used for image sharpening because it reveals more detail compared to the unmask method. Sharpening the image allows for the extraction of more precise and detailed features.

3. **Otsu Thresholding**: Otsu thresholding technique provides smoother edges and better lung segment isolation, which is why it's used in this pipeline.

4. **Edge Detection**: For edge detection, the Sobel filter is chosen as it extracts edges more effectively than the Canny filter in similar [projects](https://doi.org/10.1038/s41551-021-00787-w).

5. **Moment Calculation**: Once the lung segment is identified, the center of the moment is calculated as a feature for prediction. As many X-ray images do not have the same dimensions, this feature can pose a problem however.

6. **Rotation and Scale Invariance**: Some images may present subjects in slightly rotated positions or different sizes. Therefore, the center of the moment is needed to be invariant to rotation and scale. Hu moments are chosen for this purpose, and to make comparison easier, the moments are logged. The third moment, which depends on the other moments, and the seventh moment, which distinguishes mirror images, are dropped, as no flipped images were observed in the dataset.

The adopted pipeline is seen in the following picture:

<div style="text-align:center">
<img src="https://i.imgur.com/5d3W9p5.png" alt="workflow" width="500" height="600"/>
</div>

The selected features for building a classifier for pneumonia detection, therefore, are:
* Mean and Standard Deviation of unenhanced image
* Area of opacity
* Perimeter of visible lung regions
* Irregularity index
* Equivalent diameter
* Hu moments (5 out of 7)

The same classifiers will be used as previously, and the best vanilla model will be parameter-tuned.

In [ ]:
def area(img):
    # binarized image as input
    return np.count_nonzero(img)

def perimeter(img):
    # edges of the image as input
    return np.count_nonzero(img)

def irregularity(area, perimeter):
    # area and perimeter of the image as input, also called compactness
    I = (4 * np.pi * area) / (perimeter ** 2)
    return I

def equiv_diam(area):
    # area of image as input
    ed = np.sqrt((4 * area) / np.pi)
    return ed

def get_hu_moments(contour):
    # hu moments except 3rd and 7th (5 values)
    M = cv2.moments(contour)
    hu = cv2.HuMoments(M).ravel().tolist()
    del hu[2], hu[-1]
    log_hu = [-np.sign(a)*np.log10(np.abs(a)) for a in hu]
    return log_hu

In [ ]:
def extract_features(img):
    """
    The function  carries out the steps mentioned above on an input image.
    It begins with basic statistical calculations (mean and standard deviation),
    applies image processing techniques such as histogram equalisation,
    sharpening, thresholding, and edge detection.
    Then, it finds contours and selects the one with the most points.
    The function then calculates various features such as area, perimeter,
    irregularity, equivalent diameter, and Hu moments from the selected contour.
    It then returns these calculated features for the given input image.
    """
    mean = img.mean()
    std_dev = img.std()
    
    # Histogram equalisation
    equalized = cv2.equalizeHist(img)
    
    # Sharpening
    hpf_kernel = np.full((3, 3), -1)
    hpf_kernel[1,1] = 9
    sharpened = cv2.filter2D(equalized, -1, hpf_kernel)
    
    # Thresholding
    ret, binarized = cv2.threshold(cv2.GaussianBlur(sharpened, 
                                        (7, 7), 0), 0, 255, 
                                   cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Edge detection
    edges = skimage.filters.sobel(binarized)
    
    # Moments from contours
    contours, hier = cv2.findContours((edges * 255).astype('uint8'), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    select_contour = sorted(contours, key=lambda x: x.shape[0], reverse=True)[0]
    
    # Feature extraction
    ar = area(binarized)
    per = perimeter(edges)
    irreg = irregularity(ar, per)
    eq_diam = equiv_diam(ar)
    hu = get_hu_moments(select_contour)
    
    return [mean, std_dev, ar, per, irreg, eq_diam, *hu]

In [ ]:
# the directories that contain the train and validation images set
paths = ['../input/chest-xray-pneumonia/chest_xray/train/', 
         '../input/chest-xray-pneumonia/chest_xray/val/']

X_train = []

for trainpath in paths:
    for folder in  os.listdir(trainpath) : 
        files = gb.glob(pathname= str( trainpath + folder + '/*.jpeg'))
        for file in files: 
            image = cv2.imread(file)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # feature extraction
            features = extract_features(image)
            X_train.append(features)

X_train = np.asarray(X_train)
X_train = X_train.astype(np.float32)
np.save('X_train_fe', X_train)
del X_train

In [ ]:
#the directory that contain the test images set
testpath='../input/chest-xray-pneumonia/chest_xray/test/'

X_test = []
for folder in  os.listdir(testpath) : 
    files = gb.glob(pathname= str( testpath + folder + '/*.jpeg'))
    for file in files: 
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
        # feature extraction
        features = extract_features(image)
        X_test.append(features)

X_test = np.asarray(X_test)
X_test = X_test.astype(np.float32)
np.save('X_test_fe',X_test)
del X_test

In [ ]:
X_train = np.load('./X_train_fe.npy')
X_test = np.load('./X_test_fe.npy')
y_train = np.load('./y_train.npy')
y_test = np.load('./y_test.npy')

In [ ]:
#Scaling
sc = StandardScaler(copy=False)
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
#fitting each model using X_train and y_train
log_reg.fit(X_train, y_train)
dtc.fit(X_train, y_train)
rfc.fit(X_train, y_train)
svm.fit(X_train, y_train)

In [ ]:
# Convert the dictionary to a pandas DataFrame
df_scores = evaluate_classifiers(X_test, y_test)
print(df_scores)

In [ ]:
mcc_scorer = make_scorer(matthews_corrcoef)

# Defining parameter range
param_grid_svm = {'C': [0.1, 1, 10],  
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf', 'linear', 'poly']} 

grid_svm = GridSearchCV(svm, param_grid_svm, refit = True,
                        verbose = 3, scoring=mcc_scorer)
  
# Fitting the model for grid search on the training data
grid_svm.fit(X_train, y_train)

# Inspect the best parameters found by GridSearchCV
print('Best parameters for SVM:', grid_svm.best_params_)

In [ ]:
# Get the best model
best_svm = grid_svm.best_estimator_

# Calculate and print the metrics
print_metrics(best_svm.predict(X_train), y_train, best_svm.predict(X_test), y_test)

In [ ]:
plot_confustion_matrix(best_svm.predict(X_test), y_test, 'Optimised SVM')

## Best ML Intelligent System

In [ ]:
feats = list(extract_features(img))

pred = gb.predict([feats])

if pred == 1:
    print('Patient is infected with pneumonia')
else:
    print('Patient is normal')